In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=2048)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [3]:
question = '''为了还原用户UI操作流程，需要识别哪些UI操作是相同的，哪些是不同的。比如，提交报销单，两个人填写的报销单可能只有姓名、部门、时间这些内容是相似的，
其他的比如出差事由、时间范围等等相差非常大，在页面上看起来也很不一样。但是，它们同属于一个业务操作，应该判断相同操作，即使对应的窗口/应用从视觉上和内容上来看相差非常大。
仅用图片相似性无法解决这个问题，因为文字内容变化难以检测；仅用文字比较也不行，因为内容相差很大的UI操作可能属于同一个业务操作。
'''

solution = '''使用CV算法识别窗口/应用的区域，同一个窗口的区域分布应该是完全一样的。另外，再提取一两个关键标签，代表当前操作的内容。只要窗口区域相似，
关键标签相同，就可以判断两个UI操作属于同一个业务操作。识别窗口区域的方法是canny，通过边框识别窗口的主要布局，同一个应用内只有有限的窗口，每个窗口的布局是很少变化的。
提交关键标签，是指使用OCR识别窗口中的所有文字，然后提取关键位置的文字，比如窗口标题栏中的文字。两者结合就可以较有效地区分不同窗口，进而判断窗口的相似性。
'''

In [4]:
template_background = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据以下要解决的问题及解决方案，起草专利的背景介绍部分。
专利的背景介绍要包含所解决问题所在的技术领域介绍，问题的简单介绍，以及这个问题所产生的不良影响。
要解决的问题: {question}
提供的解决方案: {solution}
请生成这个专利发明点的背景介绍：'''

prompt_background = PromptTemplate(input_variables=['question', 'solution'], template=template_background)
background_chain = LLMChain(llm=llm, prompt=prompt_background, output_key='background')

In [5]:
template_claims = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据以下要解决的问题及解决方案，起草专利的权利要求部分。
在设计专利的权利要求时要一步步来，先判断解决方案的创新性和价值所在，然后再根据创新点来设计对应的权利要求。针对要解决的问题，先将解决方案重新描述为多个步骤，再判断每一个步骤
是否有必要存在，若是没有必要，则可以删除；对于必要存在的步骤，再考虑它是否具备创新性。若是不具备，则更新对应的步骤。只要其中一个步骤具备创新性即可。

要解决的问题: {question}
提供的解决方案: {solution}
请生成这个专利发明点的权利要求：'''

prompt_claims = PromptTemplate(input_variables=['question', 'solution'], template=template_claims)
claims_chain = LLMChain(llm=llm, prompt=prompt_claims, output_key='claims')

In [6]:
template_explanation = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据解决的问题、解决方案及权利要求，
扩充权利要求部分的内容。扩充的方法是，先设计一个具体的应用场景，将权利要求转换为具体的实现步骤，并结合应用场景进行描述每一步的输入输出，以及中间的处理过程。
若是不同权利要求对应的操作步骤是相同的，则只需要在第一次给出描述，后面的则省略。

要解决的问题: {question}
提供的解决方案: {solution}
独立权利要求: {claims}

请将权利要求扩展为具体的实现步骤，并给出相应的示例说明：'''

prompt_explanation = PromptTemplate(input_variables=['question', 'solution', 'claims'], template=template_explanation)
explanation_chain = LLMChain(llm=llm, prompt=prompt_explanation, output_key='explanation')

In [7]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[background_chain, claims_chain, explanation_chain],
    input_variables=["question", "solution"],
    # Here we return multiple variables
    output_variables=["background", "claims", "explanation"],
    verbose=True)

In [8]:
result = overall_chain({'question': question, 'solution': solution})



> Entering new SequentialChain chain...

> Finished chain.


In [9]:
print(result['background'])

本发明涉及技术领域为流程挖掘和任务挖掘，特别是涉及识别UI操作流程中相同和不同的操作。在用户UI操作流程中，有些操作可能看起来完全不同，但实际上属于同一个业务操作。这个问题对于流程挖掘和任务挖掘来说是非常重要的，因为只有准确识别每个业务操作，才能进行有效的流程挖掘和任务挖掘。然而，目前使用图片相似性或文字比较等方法无法有效地解决这个问题，因为文字内容变化难以检测，而内容相差很大的UI操作可能属于同一个业务操作，因此需要一种新的解决方案。

本发明提供了一种使用CV算法识别窗口/应用的区域，同时提取关键标签来判断UI操作属于同一个业务操作的方法。具体来说，本发明使用canny算法识别窗口的边框，通过边框识别窗口的主要布局，同一个应用内只有有限的窗口，每个窗口的布局是很少变化的。同时，本发明还提取一两个关键标签，代表当前操作的内容，比如窗口标题栏中的文字。只要窗口区域相似，关键标签相同，就可以判断两个UI操作属于同一个业务操作。

本发明的解决方案可以有效地解决识别UI操作流程中相同和不同的操作的问题。本发明的解决方案可以准确地识别每个业务操作，从而实现有效的流程挖掘和任务挖掘。同时，本发明的解决方案具有较高的实用性和广泛的应用前景，可以应用于各种UI操作流程识别和分析的场景中。


In [10]:
print(result['claims'])

1. 一种UI操作识别方法，包括以下步骤：
a. 通过边框识别窗口的主要布局；
b. 提取窗口中的所有文字；
c. 提取关键位置的文字，比如窗口标题栏中的文字；
d. 对比窗口区域分布和关键标签，判断两个UI操作是否属于同一个业务操作。

2. 根据权利要求1所述的UI操作识别方法，实现的一种系统。

3. 根据权利要求1所述的UI操作识别方法，实现的一种计算机程序产品。

4. 根据权利要求1所述的UI操作识别方法，应用于业务流程还原的一种方法。

5. 根据权利要求1所述的UI操作识别方法，应用于用户行为分析的一种方法。

6. 根据权利要求1所述的UI操作识别方法，应用于UI自动化测试的一种方法。

7. 根据权利要求1所述的UI操作识别方法，应用于UI界面优化的一种方法。


In [11]:
print(result['explanation'])

1. 一种UI操作识别方法，包括以下步骤：
a. 通过边框识别窗口的主要布局；
b. 使用OCR技术提取窗口中的所有文字；
c. 根据关键词提取关键位置的文字，比如窗口标题栏中的文字；
d. 对比窗口区域分布和关键标签，判断两个UI操作是否属于同一个业务操作。

例如，对于一个报销单页面，通过边框识别出页面中的各个区域，如报销人信息、报销单信息、费用明细等。然后使用OCR技术提取出页面中所有的文字信息，包括姓名、部门、报销单号、费用明细等。再根据关键词提取出报销单号、报销人姓名等关键信息。最后，通过对比不同报销单页面中的区域分布和关键标签，判断它们是否属于同一个业务操作。

2. 根据权利要求1所述的UI操作识别方法，实现的一种系统。
该系统包括：边框识别模块、OCR文字提取模块、关键词提取模块和UI操作判断模块。

例如，该系统可以应用于公司的财务管理系统，通过对财务系统中的各个页面进行识别和判断，帮助公司实现财务流程的自动化管理。

3. 根据权利要求1所述的UI操作识别方法，实现的一种计算机程序产品。
该计算机程序产品包括：边框识别程序、OCR文字提取程序、关键词提取程序和UI操作判断程序。

例如，该计算机程序产品可以应用于各种企业管理系统中，帮助企业实现自动化流程管理和数据分析。

4. 根据权利要求1所述的UI操作识别方法，应用于业务流程还原的一种方法。
该方法包括以下步骤：
a. 识别不同业务操作的UI页面；
b. 提取页面中的关键信息；
c. 判断不同页面是否属于同一个业务操作；
d. 还原业务流程中的UI操作流程。

例如，该方法可以应用于银行的贷款审批流程中，通过识别不同的审批页面，提取页面中的关键信息，判断不同页面是否属于同一个贷款审批流程，最终还原出贷款审批的UI操作流程。

5. 根据权利要求1所述的UI操作识别方法，应用于用户行为分析的一种方法。
该方法包括以下步骤：
a. 识别用户在系统中的UI操作流程；
b. 统计用户的操作行为数据；
c. 分析用户的操作行为数据，找出用户的行为模式和偏好；
d. 根据用户的行为模式和偏好，优化系统的UI设计。

例如，该方法可以应用于电商平台中，通过识别用户在平台中的UI操作流程，统计用户的操作行为数据，分析用户的行为模式和偏好，从而优化平台的UI设计，提高用户的购物体验。

6. 根据权利要求1所

In [12]:
with open('patent-output2.txt', 'w') as f:
    f.write('background: ' + result['background'])
    f.write('\n\nclaims: ' + result['claims'])
    f.write('\n\nexplanation: ' + result['explanation'])